In [ ]:
import torch

from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

from src.dataset import WhitespaceDataset
from src.model import ByT5WhitespaceRestorer
from src.trainer import train_and_validate


# Поменять на свои пути
PATH_TO_TEST = "/kaggle/input/whitespace-restore/test.txt"
PATH_TO_TRAIN = "/kaggle/input/whitespace-restore/train.txt"
PATH_TO_SAVE = "/kaggle/working/"

In [ ]:
test_dataset = WhitespaceDataset(PATH_TO_TEST)
train_dataset = WhitespaceDataset(PATH_TO_TRAIN)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=296,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ByT5WhitespaceRestorer()
model.to(device)

In [ ]:
epochs = 1

optimizer = AdamW(
    model.parameters(),
    lr=5e-5,
    weight_decay=0.01,
    eps=1e-8,
    fused=True
)

total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps * 0.01),
    num_training_steps=total_steps
)

train_and_validate(model, train_loader, val_loader, optimizer, scheduler, device=device, save_dir=PATH_TO_SAVE, epochs=epochs)